In [1]:
import random
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.nn.functional import one_hot
from torch.utils.data import Dataset, TensorDataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler, random_split, Subset
from torchvision.transforms import Compose, ToTensor, Normalize, ToPILImage, \
                                    RandomHorizontalFlip, Resize, transforms, RandomCrop, RandomRotation
from torchvision.datasets import ImageFolder

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

PATH = "./data/dataset"
torch.manual_seed(13)

In [2]:
torch.cuda.empty_cache()

In [3]:
%run -i dataprep_v1.py

Generated train_loader and val_loader


In [4]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# dataset = ImageFolder(root=PATH, transform=transform)

In [5]:
# labels = [label for _, label in dataset]
# one_hot_labels = one_hot(torch.tensor(labels), num_classes=len(dataset.classes)).float()

In [6]:
# for i in range(len(dataset)):
    # dataset.samples[i] = (dataset.samples[i][0], one_hot_labels[i])

In [7]:
# from sklearn.model_selection import train_test_split

# total_samples = len(dataset)
# indices = list(range(total_samples))
# train_indices, val_indices = train_test_split(indices, test_size=0.15, random_state=42)

In [8]:
# train_sampler = Subset(dataset, train_indices)
# val_sampler = Subset(dataset, val_indices)

# train_loader = DataLoader(train_sampler, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_sampler, batch_size=32, shuffle=False)

In [9]:
train_loader.dataset[0]

(tensor([[[2.0605, 1.9235, 1.8893,  ..., 2.2489, 2.2489, 2.2489],
          [1.9749, 1.5982, 1.6667,  ..., 2.2147, 2.2318, 2.2318],
          [1.9578, 1.5468, 1.6495,  ..., 2.2147, 2.2318, 2.2318],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],
 
         [[2.2360, 2.0959, 2.0434,  ..., 2.4286, 2.4286, 2.4286],
          [2.1485, 1.7458, 1.8158,  ..., 2.3936, 2.4111, 2.4111],
          [2.1310, 1.6933, 1.7983,  ..., 2.3936, 2.4111, 2.4111],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],
 
         [[2.4831, 2.3786, 2.3437,  ..., 2.6400, 2.6400, 2.6400],
          [2.3960, 2.0474, 2.1171,  ..., 2.6051, 2.6226, 2.6226],
          [2.3786, 1.9951, 2.0997,  ...,

In [10]:
#MODEL
from torchvision.models.resnet import resnet101, ResNet101_Weights, resnet152, ResNet152_Weights, resnet50, ResNet50_Weights

class ResnetCustom(nn.Module):
    def __init__(self, num_classes, p=.5) -> None:
        super(ResnetCustom, self).__init__()
        self.resnet = resnet101(weights=ResNet101_Weights.DEFAULT)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=p),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        x = self.resnet(x)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 151
resnet_v1 = ResnetCustom(num_classes, p=.6)
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(resnet_v1.resnet.fc.parameters(), lr=3e-4)
resnet_v1.to(device=device)

ResnetCustom(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [11]:
# training loop
n_epochs = 50
train_losses = []
val_losses = []

for epoch in range(n_epochs):

    batch_losses = []
    batch_val_losses = []

    for x_batch, y_batch in train_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        resnet_v1.train()

        yhat = resnet_v1(x_batch)
        loss = multi_loss_fn(yhat, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        batch_losses.append(loss.item())
    
    train_epoch_loss = np.mean(batch_losses)
    train_losses.append(train_epoch_loss)

    with torch.no_grad():

        for x_batch, y_batch in val_loader:

            resnet_v1.eval()

            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            yhat = resnet_v1(x_batch)
            loss = multi_loss_fn(yhat, y_batch)

            batch_val_losses.append(loss.item())

    val_epoch_loss = np.mean(batch_val_losses)
    val_losses.append(val_epoch_loss)

    print(f"Epoch {epoch+1}/{n_epochs}: loss = {train_epoch_loss}, val_loss = {val_epoch_loss}")
            


Epoch 1/50: loss = 4.392386341523268, val_loss = 3.832514100155588
Epoch 2/50: loss = 3.2773412009199223, val_loss = 3.1841559692964716
Epoch 3/50: loss = 2.7013283475430425, val_loss = 2.6371891700615318
Epoch 4/50: loss = 2.2638690753611264, val_loss = 2.5294660228793906
Epoch 5/50: loss = 1.9555958281020205, val_loss = 2.4490700697494767
Epoch 6/50: loss = 1.7572329276336167, val_loss = 2.315566052824764
Epoch 7/50: loss = 1.6383709878978614, val_loss = 2.6800559193401012
Epoch 8/50: loss = 1.4929499156817705, val_loss = 1.6218186859357155
Epoch 9/50: loss = 1.3766454365439045, val_loss = 1.5458102185847395
Epoch 10/50: loss = 1.2709846933801732, val_loss = 1.4780287085953405
Epoch 11/50: loss = 1.230098166330132, val_loss = 1.4751940662578
Epoch 12/50: loss = 1.168363815474653, val_loss = 1.8651190167766507
Epoch 13/50: loss = 1.114517745857467, val_loss = 1.1847111097836898
Epoch 14/50: loss = 1.105502686011577, val_loss = 1.1792245820417242
Epoch 15/50: loss = 1.0123606837259795,